In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_10%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class
0,1.49,0.0,NaN,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,NaN,0.01,0.0,0.00,0.0,0.00,23.0,1
1,0.73,0.0,0.35,0.38,NaN,5.86,0.64,2.06,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1
2,1.22,0.0,0.73,0.67,NaN,8.72,0.88,4.35,5.17,NaN,...,211.44,1431.0,NaN,0.01,0.0,10.66,1.0,0.18,23.0,1
3,2.50,0.0,1.16,NaN,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1
4,NaN,0.0,1.72,NaN,5.15,9.11,1.04,3.10,4.93,12.82,...,NaN,117.0,114.0,0.00,0.0,18.18,NaN,13.32,23.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0
1036,1.76,0.0,NaN,0.69,2.43,NaN,0.74,2.72,3.93,NaN,...,143.80,47.0,44.0,0.01,0.0,NaN,NaN,25.79,1.0,0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,NaN,...,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0,0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         110
Jitter (local, absolute)               119
Jitter (rap)                           110
Jitter (ppq5)                          112
Jitter (ddp)                           112
Shimmer (local)                        106
Shimmer (local, dB)                    101
Shimmer (apq3)                         100
Shimmer (apq5)                          99
Shimmer (apq11)                         94
Shimmer (dda)                          106
AC                                     110
NTH                                    110
HTN                                    111
Median pitch                           119
Mean pitch                             112
Standard deviation                     103
Minimum pitch                          125
Maximum pitch                          119
Number of pulses                       102
Number of periods                      111
Mean period                            113
Standard deviation of period           100
Fraction of

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 2912
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         10.576923
Jitter (local, absolute)               11.442308
Jitter (rap)                           10.576923
Jitter (ppq5)                          10.769231
Jitter (ddp)                           10.769231
Shimmer (local)                        10.192308
Shimmer (local, dB)                     9.711538
Shimmer (apq3)                          9.615385
Shimmer (apq5)                          9.519231
Shimmer (apq11)                         9.038462
Shimmer (dda)                          10.192308
AC                                     10.576923
NTH                                    10.576923
HTN                                    10.673077
Median pitch                           11.442308
Mean pitch                             10.769231
Standard deviation                      9.903846
Minimum pitch                          12.019231
Maximum pitch                          11.442308
Number of pulses                        9.807692
Number of periods   

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,5)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.49,0.0,0.00,0.79,2.70,8.33,0.78,4.52,4.61,6.80,...,142.23,187.58,160.0,0.0,0.01,0.0,0.00,0.0,0.00,23.0
1,0.73,0.0,0.35,0.38,0.00,5.86,0.64,2.06,3.18,7.19,...,159.52,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0
2,1.22,0.0,0.73,0.67,0.00,8.72,0.88,4.35,5.17,0.00,...,0.00,211.44,1431.0,0.0,0.01,0.0,10.66,1.0,0.18,23.0
3,2.50,0.0,1.16,0.00,3.47,13.51,0.00,5.26,8.77,16.78,...,182.71,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0
4,0.00,0.0,1.72,0.00,5.15,9.11,1.04,3.10,4.93,12.82,...,182.82,0.00,117.0,114.0,0.00,0.0,18.18,0.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,96.15,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0
1036,1.76,0.0,0.00,0.69,2.43,0.00,0.74,2.72,3.93,0.00,...,121.49,143.80,47.0,44.0,0.01,0.0,0.00,0.0,25.79,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,10.14,14.24,...,0.00,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,0.00,...,90.05,451.37,75.0,70.0,0.01,0.0,16.67,3.0,19.58,1.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 5922


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[0.0, 2.68, 2.16, 4.34, 2.57]
mean :  2.35
standar deviasi :  1.3911146609823362
fuzzy :  [(0.0, 0.24009822633285982), (2.68, 0.9722583580478603), (2.16, 0.990717120201178), (4.34, 0.3594896788803032), (2.57, 0.9875739777501826)]
fuzzy :  None
pembilang :  8.843851708361296
penyebut :  3.550137361212384
rata-rata tertimbang :  2.4911294433241564
 
[2.11, 2.29, 1.29, 2.5, 2.72]
mean :  2.1820000000000004
standar deviasi :  0.49052624802348754
fuzzy :  [(2.11, 0.9892865679797173), (2.29, 0.976056039598833), (1.29, 0.19143284651980577), (2.5, 0.8104919548072613), (2.72, 0.5480426252021607)]
fuzzy :  None
pembilang :  8.08641718869711
penyebut :  3.5153100341077783
rata-rata tertimbang :  2.300342533158537
 
[0.0, 1.63, 2.83, 1.16, 0.41]
mean :  1.206
standar deviasi :  0.990688649374767
fuzzy :  [(0.0, 0.4766964010196582), (1.63, 0.9124918406931186), (2.83, 0.26094293709586136), (1.16, 0.9989227109981813), (0.41, 0.7241492290261683)]
fuzzy :  None
pembilang :  3.6814817409696903

[3.83, 2.87, 1.4159739308273578, 1.86, 2.3]
mean :  2.455194786165472
standar deviasi :  0.8391036864073594
fuzzy :  [(3.83, 0.26130349249494533), (2.87, 0.8849935359216086), (1.4159739308273578, 0.4644746793089162), (1.86, 0.7776001486492269), (2.3, 0.9830433678278426)]
fuzzy :  None
pembilang :  7.90574388427308
penyebut :  3.37141522420254
rata-rata tertimbang :  2.344933316881213
 
[0.0, 2.69, 2.31, 3.04, 0.0]
mean :  1.6079999999999999
standar deviasi :  1.3330776421499235
fuzzy :  [(0.0, 0.4831531643697985), (2.69, 0.7193852190234488), (2.31, 0.8705414342791742), (3.04, 0.5616370578333694), (0.0, 0.4831531643697985)]
fuzzy :  None
pembilang :  5.653473608171412
penyebut :  3.117870039875589
rata-rata tertimbang :  1.8132486395735083
 
[2.68, 0.39, 2.4911294433241564, 0.89, 1.74]
mean :  1.6382258886648313
standar deviasi :  0.8878069150370637
fuzzy :  [(2.68, 0.5023838632344407), (0.39, 0.37222118873433513), (2.4911294433241564, 0.6303936457643675), (0.89, 0.7011017730251772), (1

[0.0, 3.46, 4.24, 3.44, 3.62]
mean :  2.9520000000000004
standar deviasi :  1.50437229434738
fuzzy :  [(0.0, 0.14586522285072565), (3.46, 0.9445857762522196), (4.24, 0.693172475922451), (3.44, 0.9487516007113513), (3.62, 0.9061276361570595)]
fuzzy :  None
pembilang :  12.751205633079476
penyebut :  3.6385027118938074
rata-rata tertimbang :  3.5045200300105286
 
[0.0, 3.19, 1.22, 2.2, 2.24]
mean :  1.7700000000000002
standar deviasi :  1.0823677748344138
fuzzy :  [(0.0, 0.2626401544490266), (3.19, 0.4229496496039471), (1.22, 0.8788927601260897), (2.2, 0.9241264275742754), (2.24, 0.910037518032261)]
fuzzy :  None
pembilang :  6.493020730646091
penyebut :  3.3986465097856
rata-rata tertimbang :  1.9104725107336027
 
[0.0, 0.99, 1.54, 2.25, 1.26]
mean :  1.208
standar deviasi :  0.7355922783716534
fuzzy :  [(0.0, 0.259682969946285), (0.99, 0.9570400679360876), (1.54, 0.9031722634420183), (2.25, 0.366705325334969), (1.26, 0.9975047450007394)]
fuzzy :  None
pembilang :  4.420297913662047
pen

[0.0, 1.18, 1.45, 1.33, 3.04]
mean :  1.4
standar deviasi :  0.9700927790680642
fuzzy :  [(0.0, 0.3530135749810228), (1.18, 0.9746152808989454), (1.45, 0.9986727581386207), (1.33, 0.9974002632669928), (3.04, 0.23958469126145485)]
fuzzy :  None
pembilang :  4.653001342341678
penyebut :  3.5632865685470367
rata-rata tertimbang :  1.3058173270186864
 
[0.0, 1.67, 4.24, 3.2, 3.85]
mean :  2.5919999999999996
standar deviasi :  1.5643963692108214
fuzzy :  [(0.0, 0.25348128505301265), (1.67, 0.8405848590938184), (4.24, 0.5741802389454999), (3.2, 0.9272649659940541), (3.85, 0.723763725061285)]
fuzzy :  None
pembilang :  9.592039160482516
penyebut :  3.3192750741476704
rata-rata tertimbang :  2.8897994128870375
 
[0.0, 3.59, 2.06, 1.07, 2.82]
mean :  1.9080000000000001
standar deviasi :  1.2670185476148326
fuzzy :  [(0.0, 0.3218254371168359), (3.59, 0.41433882992363424), (2.06, 0.9928305686518945), (1.07, 0.803563446110965), (2.82, 0.7718004953695466)]
fuzzy :  None
pembilang :  6.5689976551296

[0.0, 3.2, 2.05, 1.32, 1.73]
mean :  1.6600000000000001
standar deviasi :  1.0392112393541555
fuzzy :  [(0.0, 0.2792478017618527), (3.2, 0.3335742552264437), (2.05, 0.9320097775512284), (1.32, 0.9478917684702086), (1.73, 0.9977342034785129)]
fuzzy :  None
pembilang :  5.9553549671031405
penyebut :  3.4904578064882466
rata-rata tertimbang :  1.7061816235202767
 
[0.0, 1.71, 3.86, 2.43, 0.5132120065981902]
mean :  1.702642401319638
standar deviasi :  1.378189452133416
fuzzy :  [(0.0, 0.4662413807444085), (1.71, 0.9999857512677366), (3.86, 0.2937450081945773), (2.43, 0.8700082988773834), (0.5132120065981902, 0.6890930195572579)]
fuzzy :  None
pembilang :  5.311602343870726
penyebut :  3.3190734586413635
rata-rata tertimbang :  1.6003268412278493
 
[0.0, 2.38, 1.06, 1.04, 3.51]
mean :  1.598
standar deviasi :  1.2183332877336972
fuzzy :  [(0.0, 0.42312198536838996), (2.38, 0.8138566081320666), (1.06, 0.9071119761013048), (1.04, 0.9004394237616207), (3.51, 0.2919089972026228)]
fuzzy :  None

...

[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0,

[1.0, 1.0, 0.0, 1.0, 0.0]
mean :  0.6
standar deviasi :  0.48989794855663565
fuzzy :  [(1.0, 0.7165560753439463), (1.0, 0.7165560753439463), (0.0, 0.4724032869055458), (1.0, 0.7165560753439463), (0.0, 0.4724032869055458)]
fuzzy :  None
pembilang :  2.1496682260318387
penyebut :  3.0944747998429305
rata-rata tertimbang :  0.6946795062415605
 
[40.0, 46.0, 40.0, 46.0, 40.0]
mean :  42.4
standar deviasi :  2.9393876913398134
fuzzy :  [(40.0, 0.7165560753439465), (46.0, 0.47240328690554534), (40.0, 0.7165560753439465), (46.0, 0.47240328690554534), (40.0, 0.7165560753439465)]
fuzzy :  None
pembilang :  129.44783143658375
penyebut :  3.09447479984293
rata-rata tertimbang :  41.831922962550635
 
[31.0, 40.0, 0.0, 1.0, 1.0]
mean :  14.6
standar deviasi :  17.304334717058612
fuzzy :  [(31.0, 0.638228805340913), (40.0, 0.3405593425081195), (0.0, 0.7005467573615027), (1.0, 0.7343194318759271), (1.0, 0.7343194318759271)]
fuzzy :  None
pembilang :  34.87610552964493
penyebut :  3.1479737689623897
r

[0.0, 1.0, 32.0, 1.0, 1.0]
mean :  7.0
standar deviasi :  12.505998560690786
fuzzy :  [(0.0, 0.8550174727690435), (1.0, 0.8912969934285407), (32.0, 0.13562322100693466), (1.0, 0.8912969934285407), (1.0, 0.8912969934285407)]
fuzzy :  None
pembilang :  7.013834052507532
penyebut :  3.6645316740616005
rata-rata tertimbang :  1.9139782859984715
 
[0.0, 12.491402942817666, 1.0, 11.0, 55.0]
mean :  15.898280588563534
standar deviasi :  20.1953386657236
fuzzy :  [(0.0, 0.7335718937045473), (12.491402942817666, 0.9858730009968455), (1.0, 0.7617940924705378), (11.0, 0.9710172799434438), (55.0, 0.1534787146543351)]
fuzzy :  None
pembilang :  32.19925038373333
penyebut :  3.605734981769709
rata-rata tertimbang :  8.930010260468396
 
[0.0, 32.0, 1.0, 1.0, 1.0]
mean :  7.0
standar deviasi :  12.505998560690786
fuzzy :  [(0.0, 0.8550174727690435), (32.0, 0.13562322100693466), (1.0, 0.8912969934285407), (1.0, 0.8912969934285407), (1.0, 0.8912969934285407)]
fuzzy :  None
pembilang :  7.013834052507532

[0.0, 1.0, 1.0, 12.0, 11.0]
mean :  5.0
standar deviasi :  5.329165037789691
fuzzy :  [(0.0, 0.6439751056887527), (1.0, 0.7545294764507768), (1.0, 0.7545294764507768), (12.0, 0.4220689412492548), (11.0, 0.530605165819481)]
fuzzy :  None
pembilang :  12.410543071906902
penyebut :  3.105708165659042
rata-rata tertimbang :  3.9960429022710002
 
[0.0, 1.0, 11.0, 3.7639516931527033, 11.0]
mean :  5.352790338630541
standar deviasi :  4.772955275334241
fuzzy :  [(0.0, 0.5332327031877554), (1.0, 0.6598114265020696), (11.0, 0.49664931285311703), (3.7639516931527033, 0.9461066214703485), (11.0, 0.49664931285311703)]
fuzzy :  None
pembilang :  15.147195929056949
penyebut :  3.1324493768664077
rata-rata tertimbang :  4.835575649177664
 
[0.0, 23.0, 1.0, 1.0, 1.0]
mean :  5.2
standar deviasi :  8.908422980528035
fuzzy :  [(0.0, 0.8433736006414609), (23.0, 0.13587597179053992), (1.0, 0.8948245714029482), (1.0, 0.8948245714029482), (1.0, 0.8948245714029482)]
fuzzy :  None
pembilang :  5.8096210653912

[0.0, 24.0, 0.0, 1.0, 1.0]
mean :  5.2
standar deviasi :  9.410632284814874
fuzzy :  [(0.0, 0.8584309515678517), (24.0, 0.13597606682827623), (0.0, 0.8584309515678517), (1.0, 0.9052146603458038), (1.0, 0.9052146603458038)]
fuzzy :  None
pembilang :  5.073854924570237
penyebut :  3.663267290655587
rata-rata tertimbang :  1.3850627109610143
 
[0.0, 1.0, 1.0, 1.0, 1.0]
mean :  0.8
standar deviasi :  0.4000000000000001
fuzzy :  [(0.0, 0.13536335042701728), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206)]
fuzzy :  None
pembilang :  3.5300333611588823
penyebut :  3.6653967115858994
rata-rata tertimbang :  0.9630699318305304
 
[0.0, 40.0, 40.0, 40.0, 31.0]
mean :  30.2
standar deviasi :  15.497096502248413
fuzzy :  [(0.0, 0.14977507990286398), (40.0, 0.8187886380475708), (40.0, 0.8187886380475708), (40.0, 0.8187886380475708), (31.0, 0.9986685802437466)]
fuzzy :  None
pembilang :  129.21336255326463
penyebut :  3.604809574289323
rata-r

In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
0,1.490000,0.0,0.664392,0.790000,2.700000,8.330000,0.780000,4.52,4.610000,6.800000,...,142.230000,187.58000,160.0,171.813290,0.010000,0.0,1.840882,0.0,0.00,23.0
1,0.730000,0.0,0.350000,0.380000,2.324461,5.860000,0.640000,2.06,3.180000,7.190000,...,159.520000,234.51000,170.0,169.000000,0.010000,0.0,2.250000,0.0,0.00,23.0
2,1.220000,0.0,0.730000,0.670000,0.866937,8.720000,0.880000,4.35,5.170000,4.843040,...,145.859807,211.44000,1431.0,1120.097306,0.010000,0.0,10.660000,1.0,0.18,23.0
3,2.500000,0.0,1.160000,1.271085,3.470000,13.510000,0.595983,5.26,8.770000,16.780000,...,182.710000,220.23000,94.0,92.000000,0.000369,0.0,9.402517,0.0,0.00,23.0
4,2.491129,0.0,1.720000,0.980024,5.150000,9.110000,1.040000,3.10,4.930000,12.820000,...,182.820000,234.15318,117.0,114.000000,0.000000,0.0,18.180000,0.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.130000,0.0,2.720000,2.510000,8.160000,9.850000,0.970000,4.59,6.810000,8.300000,...,96.150000,158.59000,63.0,60.000000,0.010000,0.0,24.190000,2.0,15.11,1.0
1036,1.760000,0.0,1.294677,0.690000,2.430000,8.021273,0.740000,2.72,3.930000,8.198708,...,121.490000,143.80000,47.0,44.000000,0.010000,0.0,10.732548,0.0,25.79,1.0
1037,2.910000,0.0,1.140000,1.270000,3.420000,15.490000,1.460000,6.43,10.140000,14.240000,...,98.358876,149.64000,79.0,77.000000,0.010000,0.0,20.000000,1.0,4.57,1.0
1038,3.790000,0.0,1.790000,1.480000,5.360000,15.700000,1.370000,6.37,11.090000,11.254230,...,90.050000,451.37000,75.0,70.000000,0.010000,0.0,16.670000,3.0,19.58,1.0


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000
mean,2.618254,0.0,1.204029,1.316423,3.628817,12.690573,1.169072,5.563115,7.834694,11.983413,...,132.671923,231.336421,108.149964,104.043286,0.006712,0.00001,27.754232,1.041346,10.979577,12.667808
std,1.720823,0.0,0.902049,1.107954,2.821140,5.347577,0.410167,2.886021,4.747385,5.863840,...,46.123484,119.482662,149.799734,143.487682,0.004700,0.00031,19.727615,1.580445,14.622635,15.457172
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,55.282171,73.524123,4.283888,4.000000,0.000000,0.00000,0.065366,0.000000,0.000000,0.694680
25%,1.503381,0.0,0.607674,0.670000,1.828433,9.255000,0.919048,3.667500,5.160000,7.985000,...,100.159137,143.055000,42.000000,40.000000,0.000000,0.00000,10.442542,0.000000,0.000000,1.000000
50%,2.336208,0.0,1.010000,1.081089,3.030000,11.985000,1.154698,5.050000,6.890000,11.240000,...,125.308315,193.896622,64.000000,61.000000,0.010000,0.00000,25.719541,1.000000,1.995000,5.000000
75%,3.340000,0.0,1.560000,1.630000,4.690000,15.032500,1.380000,6.715000,9.222500,14.850000,...,156.600623,260.630000,111.000000,108.250000,0.010000,0.00000,41.895000,1.000000,19.705000,23.000000
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,44.760000,...,452.080000,526.410000,1490.000000,1483.000000,0.010000,0.01000,88.160000,12.000000,69.120000,55.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[127   8]
 [  4 121]]
0.9538461538461539
